In [128]:
import numpy as np
import pandas as pd
import plotly.express as px          

In [129]:

params ={
    'fname': 'bank_statement.csv'
}

In [130]:
df = pd.read_csv(params.get('fname'), sep=';', dtype={'Sorte': object}, encoding='latin3')

In [131]:
df_cols = ['Buchungstag','Wertstellungstag','GegenIBAN','Name Gegenkonto','Verwendungszweck','Umsatz','W�hrung']

In [132]:
df.columns = df_cols
df = df.iloc[9:]
df = df.reset_index(drop=True)

In [133]:
kategorie = ['Lebensmittel','Wohnen', 'Mobilität', 'Sport', 'ETF', 'Sonstiges', 'Einkommen']

In [134]:
df['Kategorien'] = 'NaN'

df['Kategorien'] = np.where(df['Verwendungszweck'].str.contains('Bayerische Oberlandb|OBB'), 'Mobilität', df['Kategorien'])
df['Kategorien'] = np.where(df['Verwendungszweck'].str.contains('STUD.WERK'), 'Wohnen', df['Kategorien'])
df['Kategorien'] = np.where(df['Verwendungszweck'].str.contains('ALDI SUED|Netto|REWE|EDEKA'), 'Lebensmittel', df['Kategorien'])
df['Kategorien'] = np.where(df['Verwendungszweck'].str.contains('McFIT|Clever Fit|Fitmart'), 'Sport', df['Kategorien'])
df['Kategorien'] = np.where(df['Verwendungszweck'].str.contains('Luca Jakob Wagner'), 'ETF', df['Kategorien'])
df['Kategorien'] = np.where(~df['Umsatz'].str.contains('-'), 'Einkommen', df['Kategorien'])
df['Kategorien'] = np.where(df['Kategorien'].str.contains('NaN'), 'Sonstiges', df['Kategorien'])
df['Umsatz'] = df['Umsatz'].str.replace(',', '.').astype(float)

df['Kategorien'] = pd.Categorical(df['Kategorien'], categories= kategorie, ordered=True)

In [146]:
def total_expenses(df):
    expense_kategories = ['Lebensmittel','Wohnen', 'Mobilität', 'Sport', 'ETF', 'Sonstiges']
    result = {}
    for i in expense_kategories:
        result.update({i: df.loc[df['Kategorien'] == i, 'Umsatz'].sum()})
    return result

def expenses_income(df):
    result2 = {'Summe ausgaben': df.loc[df['Kategorien'] != 'Einkommen', 'Umsatz'].sum(),
              'Einkommen': df.loc[df['Kategorien'] == 'Einkommen', 'Umsatz'].sum(),
              'differenz': df['Umsatz'].sum()}
    return result2

In [143]:
expenses = total_expenses(df)

Expenses_Chart = px.bar(x = expenses.keys(), y = expenses.values(), 
                                      title = "Ausgaben vom: " + df['Wertstellungstag'].iloc[-1]
                                      + " bis " + df['Wertstellungstag'].iloc[0], color=expenses.keys())
Expenses_Chart.update_yaxes(title = 'Ausgaben (EUR)', visible = True, showticklabels = True)
Expenses_Chart.update_xaxes(title = 'Kategorie', visible = True, showticklabels = True)
Expenses_Chart.show()

In [147]:
total = expenses_income(df)

Expenses_Chart = px.bar(x = total.keys(), y = total.values(), 
                                      title = "Ausgaben und Einkommen vom: " + df['Wertstellungstag'].iloc[-1]
                                      + " bis " + df['Wertstellungstag'].iloc[0], color=total.keys())
Expenses_Chart.update_yaxes(title = 'Ausgaben/Einkommen (EUR)', visible = True, showticklabels = True)
Expenses_Chart.update_xaxes(title = 'Kategorie', visible = True, showticklabels = True)
Expenses_Chart.show()